In [109]:

import pso
import particle
import os
from utils import load_cloud_dataset, load_breast_cancer
import importlib



In [124]:
importlib.reload(particle)
importlib.reload(pso)


<module 'pso' from '/home/skholkin/projects/PSO_GMM/pso.py'>

In [127]:
config = pso.PSOConfig.from_json(os.path.join('configs', 'default_params.json'))

data = load_breast_cancer()
config.eig_val_max = 1000

pso_algo = pso.PSOEigen(data, config)

In [128]:
pso_algo.run()

KeyboardInterrupt: 